# Modélisation du problème

In [224]:
class Probleme:
    def __init__(self, max_places):
        self.max_places = max_places
        self.somme_gauche = 0
        self.somme_droite = 0
        self.nb_gauche = 0
        self.nb_droite = 0
        self.touristes = []

    def simulation(self, tirage, strategie, print_perf=True):
        self.somme_gauche = 0
        self.somme_droite = 0
        self.nb_gauche = 0
        self.nb_droite = 0
        self.touristes = []
        nb_lancers = self.max_places * 2
        for _ in range(nb_lancers) :
            valeur_a_placer = tirage()
            self.touristes.append(valeur_a_placer)
            strategie(self, valeur_a_placer)
        performance = abs(self.somme_droite - self.somme_gauche)
        if print_perf :
            print(f'Performance obtenue : {performance}')
        return performance

Avant d'aborder le bateau ... 

Je vais simplifier le problème afin de faciliter l'intuition mais aussi d'obtenir des premières résolutions.

Ces premières résolutions pourront être remises en question par la complexification du problème, mais il est possible que les raisons qui invalident ces solutions soient alors plus faciles à cerner.

Pour simplifier le problème je compte jouer sur deux paramètres : **La variable aléatoire** (matérialisée par le poids des touristes) et **le nombre de tirages** (materialisé par le nombre de places sur le bateau).

# Définition de modalités pour ces paramètres.

## Variable aléatoire

Je vais utiliser trois modalités de référence pour la loi de probabilité. Pour rester dans l'idée de poids de touristes européens, je vais utiliser des valeurs réalistes bien que fausses.

Modalité A : Loi uniforme sur [40;105] (moyenne : 72.5)

Modalité B : Loi normale de moyenne 72.5 et d'écart type 10

Modalité C : Combinaison de deux lois normales de paramètres (65, 10) et (80, 10) chacun ayant une probabilité de 0.5 d'être sélectionnée pour le tirage

La modalité A apporte du réalisme aux valeurs, la modalité B ajoute du réalisme à la distribution, et la modalité C permet  d'introduire une prise en compte de classes d'individus (par exemple homme et femmes)

In [225]:
import random

def tirage_a() :
    return random.uniform(40, 105)

def tirage_b() :
    return random.normalvariate(72.5, 10)

def tirage_c() :
    return random.normalvariate(65, 10) if random.choice(['femme', 'homme']) == 'femme' else random.normalvariate(80, 10)

## Nombre de places

Afin de mieux percevoir les enjeux de la position d'un touriste relativement à la position finale, il peux être intéressant d'observer la performance des stratégies sur un nombre plus réduit de tirages, on utilisera donc trois modalités, avec respectivement deux, trois, et quatre places de chaque coté du bateau, soit quatre, six et huit tirages.

In [226]:
probleme_deux = Probleme(max_places=2)
probleme_trois = Probleme(max_places=3)
probleme_quatre = Probleme(max_places=4)

# Définition d'un première stratégie  : Aveugle

Pour commencer je défini la stratégie aveugle, qui consiste a placer les touristes sans même tenir compte du poids observé.

L'ordre n'a aucune importance dans un tel cadre, je décide donc de remplir entièrement le coté gauche, puis entièrement le coté droit.

In [227]:
def strategie_aveugle(probleme, valeur) :
    if probleme.nb_gauche < probleme.max_places :
        probleme.nb_gauche += 1
        probleme.somme_gauche += valeur
    else :
        probleme.nb_droite += 1
        probleme.somme_droite += valeur
    return None

# Définition d'une meilleure stratégie 

Je souhaite profiter de la connaissance du poids des touristes, et du nombre de places disponsibles afin de modifier la stratégie aveugle pour mener à un gain en performance.

Pour tenir compte des places restantes, je vais considérer que les places vides sont occupées par un touriste fictif, de poids moyen. Cela représente le déroulement plus vraissemblable.

Lorsqu'un touriste arrive, il vient donc remplacer un "touriste fictif" du coté qui provoque le déséquilibre le plus faible.

In [228]:
def strategie_progressive(probleme, valeur, poids_moyen=72.5):
    # Lorsque le coté gauche est plein, le touriste se place inévitablement à droite.
    if probleme.nb_gauche == probleme.max_places :
        probleme.nb_droite += 1
        probleme.somme_droite += valeur
        return None
    # Lorsque le coté droite est plein, le touriste se place inévitablement à gauche.
    if probleme.nb_droite == probleme.max_places :
        probleme.nb_gauche += 1
        probleme.somme_gauche += valeur
        return None
    
    # Lorsqu'il reste des places des deux cotés :
    # On évalue les déséquilibres induits par un placement à droite, et par un placement à gauche.
    etat_gauche = (probleme.max_places - probleme.nb_gauche) * poids_moyen + probleme.somme_gauche
    etat_droite = (probleme.max_places - probleme.nb_droite) * poids_moyen + probleme.somme_droite
    desequilibre_placement_gauche = abs(etat_gauche + valeur - poids_moyen - etat_droite)
    desequilibre_placement_droite = abs(etat_droite + valeur - poids_moyen - etat_gauche)
    # On effectue le placement qui induit le déséquilibre le plus faible, en cas d'égalité (bateau vide) on place à gauche
    if desequilibre_placement_gauche <= desequilibre_placement_droite :
        probleme.nb_gauche += 1
        probleme.somme_gauche += valeur
    else :
        probleme.nb_droite += 1
        probleme.somme_droite += valeur
    return None

# Définition de l'optimum

Sans connaitre une hypothétique stratégie optimale, je peux définir l'optimum d'un tirage en le déterminant à posteriori. Avec la connaissance des huit touristes, on peut identifier le déséquilibre minimal qui pouvait être atteint par l'ensemble des stratégies existantes.

In [229]:
from itertools import combinations

def partage_equilibre(liste, max_place) :
    combinaisons = list(combinations(liste, max_place))

    difference_minimale = 10000
    for combinaison in combinaisons :
        liste_droite = list(combinaison)
        liste_gauche = [valeur for valeur in liste if valeur not in liste_droite]

        somme_droite = sum(liste_droite)
        somme_gauche = sum(liste_gauche)
        difference = abs(somme_droite - somme_gauche)
        if difference < difference_minimale :
            difference_minimale = difference
            
    return difference_minimale

# Evaluation des stratégies

In [230]:
import pandas as pd

tirages = [tirage_a, tirage_b, tirage_c]
problemes = [probleme_deux, probleme_trois, probleme_quatre]
combinaisons = [(tirage.__name__, probleme) for tirage in tirages for probleme in problemes]

strategies = [strategie_aveugle, strategie_progressive]

columns = [strategie.__name__ for strategie in strategies]

df_performance = pd.DataFrame(columns=columns, index=[f'{tirage[-1]} - {probleme.max_places}' for tirage, probleme in combinaisons])
df_distance_optimum = pd.DataFrame(columns=columns, index=[f'{tirage[-1]} - {probleme.max_places}' for tirage, probleme in combinaisons])

sample_size=1000

for tirage, probleme in combinaisons:
    tirage_fun = globals()[tirage]
    for strategie in strategies:
        performance_moyenne = 0
        distance_optimum_moyenne = 0
        for _ in range(sample_size) :
            performance = probleme.simulation(tirage_fun, strategie, print_perf=False)
            performance_moyenne += performance / sample_size
            distance_optimum = performance - partage_equilibre(probleme.touristes, probleme.max_places)
            distance_optimum_moyenne += distance_optimum / sample_size
        df_performance.loc[f'{tirage[-1]} - {probleme.max_places}', strategie.__name__] = performance_moyenne
        df_distance_optimum.loc[f'{tirage[-1]} - {probleme.max_places}', strategie.__name__] = distance_optimum_moyenne

In [232]:
df_performance

,strategie_aveugle,strategie_progressive
a - 2,30.906325,21.530747
a - 3,35.828643,22.411221
a - 4,42.519317,24.721257
b - 2,16.356007,11.323513
b - 3,19.020461,12.751661
b - 4,21.53451,12.855353
c - 2,20.357549,14.550555
c - 3,23.963517,16.046828
c - 4,27.302474,16.858555


In [233]:
df_distance_optimum

,strategie_aveugle,strategie_progressive
a - 2,17.853474,8.398794
a - 3,29.630789,16.288362
a - 4,40.169541,22.432838
b - 2,9.53303,4.664759
b - 3,15.855665,9.752345
b - 4,20.463318,11.645945
c - 2,11.940523,6.198305
c - 3,20.26095,12.256334
c - 4,25.946353,15.459656


J'observe une meilleure performance sur la stratégie progressive par rapport à la stratégie aveugle pour toutes les distributions. Cette performance progresse plus fortement lorsque le nombre de places sur le bateau augmente.

La distance à l'optimum diminue aussi largement entre la stratégie aveugle et la stratégie progressive. Pour toutes les modalités on divise a peu près cette distance par deux.